<a href="https://colab.research.google.com/github/aminKMT/RNN-LastMile/blob/main/LSTM_LastMile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

# Sample delivery sequence data
batch_size = 32
sequence_length = 10
feature_dim = 6
num_samples = 1000

# Simulated data
X_seq = torch.randn(num_samples, sequence_length, feature_dim)
y_eta = torch.randn(num_samples) * 2 + 5  # Delivery time around 5 hours



In [39]:
y_eta[999]

tensor(3.7376)

In [5]:
# Dataset & Dataloader
dataset = TensorDataset(X_seq, y_eta)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# LSTM Model
class ETA_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1]).squeeze(1)



In [ ]:
# Initialize model
model = ETA_LSTM(input_dim=feature_dim, hidden_dim=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()



In [ ]:
# Training loop
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

Epoch 1: Loss = 258.5162
Epoch 2: Loss = 143.4163
Epoch 3: Loss = 145.0192
Epoch 4: Loss = 143.5402
Epoch 5: Loss = 145.6227
Epoch 6: Loss = 141.2747
Epoch 7: Loss = 140.9509
Epoch 8: Loss = 139.2878
Epoch 9: Loss = 143.8311
Epoch 10: Loss = 145.9045


## LSTM with cross validation:

In [44]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
import numpy as np
import copy

# Parameters
batch_size = 32
sequence_length = 10
feature_dim = 6
num_samples = 1000
n_splits = 5
n_epochs = 10

# Simulated data
X_seq = torch.randn(num_samples, sequence_length, feature_dim)
y_eta = torch.randn(num_samples) * 2 + 5  # Around 5 hours

# LSTM Model
class ETA_LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1]).squeeze(1)

# Cross-validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
val_losses = []
best_model = None
best_val_loss = float('inf')

for fold, (train_idx, val_idx) in enumerate(kf.split(X_seq)):
    print(f"\n--- Fold {fold+1} ---")

    # Prepare DataLoaders
    X_train, X_val = X_seq[train_idx], X_seq[val_idx]
    y_train, y_val = y_eta[train_idx], y_eta[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    # Initialize model and optimizer
    model = ETA_LSTM(input_dim=feature_dim, hidden_dim=64)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.MSELoss()

    # Training loop
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            loss = loss_fn(pred, yb)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    print(f"Fold {fold+1}: Validation Loss = {avg_val_loss:.4f}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model = copy.deepcopy(model)

# Final result
print(f"\n✅ Best fold: {np.argmin(val_losses)+1}, Validation Loss = {best_val_loss:.4f}")



--- Fold 1 ---
Epoch 1: Train Loss = 233.1860
Epoch 2: Train Loss = 103.6068
Epoch 3: Train Loss = 100.3196
Epoch 4: Train Loss = 103.0483
Epoch 5: Train Loss = 101.0623
Epoch 6: Train Loss = 100.7240
Epoch 7: Train Loss = 100.9774
Epoch 8: Train Loss = 106.6424
Epoch 9: Train Loss = 100.6920
Epoch 10: Train Loss = 99.7164
Fold 1: Validation Loss = 5.0596

--- Fold 2 ---
Epoch 1: Train Loss = 249.3735
Epoch 2: Train Loss = 112.8941
Epoch 3: Train Loss = 107.0793
Epoch 4: Train Loss = 111.3128
Epoch 5: Train Loss = 106.1904
Epoch 6: Train Loss = 107.0464
Epoch 7: Train Loss = 111.8735
Epoch 8: Train Loss = 108.7336
Epoch 9: Train Loss = 105.9576
Epoch 10: Train Loss = 109.9441
Fold 2: Validation Loss = 4.0764

--- Fold 3 ---
Epoch 1: Train Loss = 204.8273
Epoch 2: Train Loss = 101.7706
Epoch 3: Train Loss = 103.5812
Epoch 4: Train Loss = 103.5988
Epoch 5: Train Loss = 102.7374
Epoch 6: Train Loss = 106.2769
Epoch 7: Train Loss = 102.5395
Epoch 8: Train Loss = 105.5044
Epoch 9: Train Lo